In [4]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 2. 고도화된 피처 엔지니어링 (포지션 가중치 강화)
def feature_engineering(df):
    # 기존 유망주 지수 로직 유지 + 보강
    df['Offensive_IQ'] = df[['Positioning', 'Vision', 'Reactions']].mean(axis=1)
    df['Technician_Score'] = df[['Dribbling', 'BallControl', 'Agility']].mean(axis=1)
    df['Physical_Potential'] = df[['Strength', 'SprintSpeed', 'Stamina']].mean(axis=1)
    
    # [새로운 지표] 나이 대비 효율성 (핵심)
    # 18세의 70점과 22세의 70점은 가치가 다름
    df['Age_Efficiency'] = df['STRating'] / (df['Age'] - 13)
    
    # 범주형 변수 처리 (Position)
    le = LabelEncoder()
    df['Position_Enc'] = le.fit_transform(df['Position'].astype(str))
    
    return df

train_df = feature_engineering(train)
test_df = feature_engineering(test)

# 학습용 피처 선정 (수치형만)
features = train_df.select_dtypes(include=[np.number]).columns.drop(['Prospect']).tolist()

X_train = train_df[features]
y_train = train_df['Prospect']
X_test = test_df[features]

# 3. 3대 모델 정의 (각기 다른 관점으로 학습)
lgb = LGBMClassifier(n_estimators=500, learning_rate=0.05, verbose=-1, random_state=42)
xgb = XGBClassifier(n_estimators=500, learning_rate=0.05, random_state=42)
cat = CatBoostClassifier(n_estimators=500, learning_rate=0.05, verbose=0, random_state=42)

# 4. 앙상블 학습 (Soft Voting 방식 직접 구현)
#print("⏳ 모델 학습 중 (LGBM, XGB, CatBoost)...")
lgb.fit(X_train, y_train)
xgb.fit(X_train, y_train)
cat.fit(X_train, y_train)

# 5. 확률 결합 (각 모델의 예측 확률 평균)
lgb_prob = lgb.predict_proba(X_test)[:, 1]
xgb_prob = xgb.predict_proba(X_test)[:, 1]
cat_prob = cat.predict_proba(X_test)[:, 1]

# 가중치 부여 (성능에 따라 조절 가능)
test_df['Prospect_Prob'] = (lgb_prob * 0.35) + (xgb_prob * 0.3) + (cat_prob * 0.35)

# 6. 공격수 TOP 10 선정
fw_positions = ['ST', 'LW', 'RW', 'CF', 'LF', 'RF']
final_top10 = test_df[test_df['Position'].isin(fw_positions)].sort_values(
    by='Prospect_Prob', ascending=False
).head(10)

print("\n" + "="*95)
print("🏆 [Ensemble Version] 차세대 공격수 유망주 TOP 10")
print("-" * 95)
output_cols = ['ID', 'Age', 'Position', 'Prospect_Prob', 'Age_Efficiency', 'Offensive_IQ']
print(final_top10[output_cols].to_string(index=False))
print("="*95)


🏆 [Ensemble Version] 차세대 공격수 유망주 TOP 10
-----------------------------------------------------------------------------------------------
       ID  Age Position  Prospect_Prob  Age_Efficiency  Offensive_IQ
TEST_1224   16       ST       0.989781        1.735167      3.543667
TEST_1066   16       ST       0.987229        1.963467      5.181267
TEST_1548   17       ST       0.984294        1.301375      4.566433
TEST_0930   17       ST       0.981037        1.267125      4.103400
TEST_1594   18       ST       0.979630        1.123280      4.486800
TEST_0523   18       ST       0.977308        1.095900      4.532933
TEST_1533   17       ST       0.975186        1.335625      4.710533
TEST_0492   17       ST       0.974054        1.267125      4.335333
TEST_0191   18       ST       0.970322        1.095900      4.601900
TEST_0139   17       ST       0.970089        1.232875      4.251433
